# 使用 DeepSeek R1 和 Ollama 实现本地 RAG 应用

https://github.com/datawhalechina/handy-ollama/tree/main/notebook/C7/DeepSeek_R1_RAG

本文档详细介绍如何利用 DeepSeek R1 和 Ollama 构建本地化的 RAG（检索增强生成）应用。同时也是对 使用 LangChain 搭建本地 RAG 应用 的补充。

我们将通过实例演示完整的实现流程，包括文档处理、向量存储、模型调用等关键步骤。  
本教程选用 DeepSeek-R1 1.5B 作为基础语言模型。考虑到不同模型具有各自的特点和性能表现，读者可以根据实际需求选择其他合适的模型来实现 RAG 系统。



## 前期准备

- 下载 Ollama https://ollama.com/download
- 验证安装 `ollama -h`
- 拉取 deepseek 模型 `ollama pull deepseek-r1:1.5b`
- 拉取  文本嵌入模型  `ollama pull nomic-embed-text`  
- 安装依赖

```
conda activate
conda activate llm-study 

# langchain_community
pip install langchain langchain_community

# Chroma
pip install langchain_chroma

# Ollama
pip install langchain_ollama

pip install pdfplumber 
```



## 文档加载

加载 PDF 文档并将其切分为适当大小的文本块。

In [3]:
from langchain_community.document_loaders import PDFPlumberLoader

file = "./data/DeepSeek_R1.pdf"

# Load the PDF
loader = PDFPlumberLoader(file)
docs = loader.load()

from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
all_splits = text_splitter.split_documents(docs)


## 初始化向量存储

使用 Chroma 数据库存储文档向量，并配置 Ollama 提供的嵌入模型。

In [4]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

local_embeddings = OllamaEmbeddings(model="nomic-embed-text")

vectorstore = Chroma.from_documents(documents=all_splits, embedding=local_embeddings)


现在，我们得到了一个向量存储，可以用来进行相似度搜索。

In [9]:
question = "DeepSeek 项目的主要目标是什么?"
docs = vectorstore.similarity_search(question)
for doc in docs:
    print(doc.page_content)

in tasks such as function calling, multi-turn, complex role-playing, and JSON output.
Movingforward,weplantoexplorehowlongCoTcanbeleveragedtoenhancetasksin
thesefields.
• LanguageMixing: DeepSeek-R1iscurrentlyoptimizedforChineseandEnglish,which
may result in language mixing issues when handling queries in other languages. For
instance,DeepSeek-R1mightuseEnglishforreasoningandresponses,evenifthequeryis
inalanguageotherthanEnglishorChinese. Weaimtoaddressthislimitationinfuture
updates.
3.1. DeepSeek-R1Evaluation
Claude-3.5- GPT-4o DeepSeek OpenAI OpenAI DeepSeek
Benchmark(Metric)
Sonnet-1022 0513 V3 o1-mini o1-1217 R1
Architecture - - MoE - - MoE
#ActivatedParams - - 37B - - 37B
#TotalParams - - 671B - - 671B
MMLU(Pass@1) 88.3 87.2 88.5 85.2 91.8 90.8
MMLU-Redux(EM) 88.9 88.0 89.1 86.7 - 92.9
MMLU-Pro(EM) 78.0 72.6 75.9 80.3 - 84.0
DROP(3-shotF1) 88.3 83.7 91.6 83.9 90.2 92.2
IF-Eval(PromptStrict) 86.5 84.3 86.1 84.8 - 83.3
English
engineeringtasks. Asaresult,DeepSeek-R1hasnotdemonstrate

## 构建 Chain 表达式



In [6]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="deepseek-r1:7b",
)

prompt = ChatPromptTemplate.from_template(
    "总结这些检索到的文档中的主要主题：{docs}"
)

# 将传入的文档转换成字符串的形式
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = {"docs": format_docs} | prompt | model | StrOutputParser()

question = "DeepSeek 项目的主要目标是什么？"

docs = vectorstore.similarity_search(question)

chain.invoke(docs)


'<think>\n嗯，我现在要帮用户总结一些文档中的主要主题。首先，我得仔细阅读用户提供的内容。看起来这是一个关于模型评估和改进的报告。\n\n第一段提到任务包括函数调用、多轮对话、复杂角色扮演和JSON输出。这些任务可能在自然语言处理中很常见，所以我要把这些作为主要主题之一。\n\n接下来，文档提到了语言混合问题。DeepSeek-R1目前优化了中文和英语，这可能导致在处理其他语言的查询时出现语言混合问题。比如，当输入不是这两种语言时，模型可能会用英文来推理和回答。我需要把这个作为另一个主题，指出这是需要解决的问题。\n\n然后是评估部分，对比了Claude-3.5、GPT-4以及DeepSeek的性能指标，如MMLU和DROP分数。看起来DeepSeek-R1在这些基准测试中表现不错，但软件工程任务上没有显著提升。这可能是因为模型更多关注通用推理能力而非特定领域。\n\n接下来，文档提到未来版本将解决软件工程数据的问题，比如拒绝采样或并行评估来提高效率。这部分是关于模型的未来改进方向，应该包括进去。\n\n最后，模型对比部分显示DeepSeek-R1-Zero在纯强化学习方面表现优异，并且通过知识蒸馏优化了小规模密集模型。这也说明了模型的不同版本和改进点。\n\n综合这些信息，我需要总结出五个主要主题：任务类型、语言混合问题、评估结果、未来改进方向以及模型对比。每个部分要简明扼要地概括关键点，这样用户就能快速抓住重点。\n</think>\n\n### 主要主题总结：\n\n1. **任务类型**  \n   - 包括函数调用、多轮对话、复杂角色扮演和JSON输出等任务。\n\n2. **语言混合问题（Language Mixing Issues）**  \n   - DeepSeek-R1目前仅优化了中文和英语，可能导致在处理其他语言查询时出现推理和响应中使用英文的问题。\n   - 需要在未来版本中解决这一限制。\n\n3. **评估结果与性能对比**  \n   - 在软件工程任务上，DeepSeek-R1未显著提升基准测试（如MMLU、DROP等）的表现。  \n   - 未来版本将通过拒绝采样或并行评估来改进在软件工程方面的效率。\n\n4. **模型改进方向**  \n   - 暂时专注于通用推理能力的优化。\n   - 在未来版本中考虑引入特定领

## 带有检索的 QA

最后，我们将检索到的相似文本段落与完整文档合并为统一的上下文（context）。随后，将用户提问（question）与这个上下文结合，按照 RAG_TEMPLATE 的格式整合，最终输入到模型中进行问答处理。

In [10]:
from langchain_core.runnables import RunnablePassthrough

RAG_TEMPLATE = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

<context>
{context}
</context>

Answer the following question:

{question}"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

retriever = vectorstore.as_retriever()

qa_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | model
    | StrOutputParser()
)

question = "What is the purpose of the DeepSeek project?"

# Run
qa_chain.invoke(question)

"<think>\nOkay, so I need to figure out what the purpose of the DeepSeek project is based on the given context. Let me read through the context carefully.\n\nThe first part mentions that DeepSeek-R1 has not shown a huge improvement over DeepSeek-V3 in software engineering benchmarks. But it's planning future versions with new techniques like rejection sampling or asynchronous evaluations to improve efficiency.\n\nThen, it says that DeepSeek-R1 avoids length bias during GPT-based evaluations and is robust across multiple tasks. On math tasks, it performs as well as OpenAI-o1-1217 but surpasses other models by a margin. It's similar on coding tasks like LiveCodeBench and Codeforces, where reasoning-focused models do better. However, for engineering-oriented coding tasks, OpenAI-o1-1217 does better than DeepSeek-R1.\n\nNext, it talks about an initial research showing that reasoning capabilities of LLMs can be incentivized through RL without needing SFT. This is a breakthrough because it p